# HN Success Predictor V3 - Stacking Meta-Learner

**Architecture:** RoBERTa (Focal Loss) + TF-IDF + Engineered Features → LightGBM Stacker → Isotonic Calibration

**Key Improvements:**
- Focal Loss (α=0.25, γ=2.0) to reduce false positives
- LightGBM meta-learner instead of weighted average
- Isotonic calibration for probability estimates
- 30+ engineered features

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q transformers datasets accelerate scikit-learn pandas tqdm lightgbm seaborn joblib

In [ ]:
import requests
import time
import json
import re
import math
import os
from datetime import datetime, timedelta
from urllib.parse import urlparse
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, precision_recall_curve, roc_curve,
    average_precision_score
)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression
from datasets import Dataset
import lightgbm as lgb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Configuration

In [ ]:
# Model configuration
MODEL_NAME = "roberta-base"
MAX_LENGTH = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 3
SEED = 42

# Focal Loss parameters (key for reducing FP)
FOCAL_ALPHA = 0.25  # Weight for positive class (lower = penalize FP more)
FOCAL_GAMMA = 2.0   # Focus on hard examples

# LightGBM stacker parameters
LGBM_PARAMS = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'is_unbalance': True,
    'verbose': -1,
    'seed': SEED
}

# Data settings
POSTS_PER_CATEGORY = 30000
MIN_SCORE_HIT = 100

np.random.seed(SEED)
torch.manual_seed(SEED)
print("✓ Configuration set")

## 3. Data Fetching

In [ ]:
def fetch_hn_posts(min_points, max_points, target_count, months_back=24):
    """Fetch HN posts from Algolia API with time-based pagination."""
    base_url = "https://hn.algolia.com/api/v1/search_by_date"
    all_posts = []
    seen_ids = set()

    if max_points:
        points_filter = f"points>={min_points},points<={max_points}"
        desc = f"{min_points}-{max_points} pts"
    else:
        points_filter = f"points>={min_points}"
        desc = f"{min_points}+ pts"

    end_date = datetime.now()
    pbar = tqdm(range(months_back), desc=f"Fetching {desc}")

    for month_offset in pbar:
        month_end = end_date - timedelta(days=30 * month_offset)
        month_start = month_end - timedelta(days=30)
        
        # Paginate within each month (up to 10 pages of 1000)
        for page in range(10):
            if len(all_posts) >= target_count:
                break
                
            params = {
                "tags": "story",
                "numericFilters": f"{points_filter},created_at_i>={int(month_start.timestamp())},created_at_i<={int(month_end.timestamp())}",
                "hitsPerPage": 1000,
                "page": page,
            }

            try:
                resp = requests.get(base_url, params=params, timeout=30)
                resp.raise_for_status()
                data = resp.json()
                hits = data.get("hits", [])
                
                if not hits:
                    break

                for hit in hits:
                    post_id = hit.get("objectID")
                    if post_id and post_id not in seen_ids:
                        seen_ids.add(post_id)
                        url = hit.get("url", "")
                        domain = ""
                        if url:
                            try:
                                domain = urlparse(url).netloc.replace("www.", "")
                            except:
                                pass

                        all_posts.append({
                            "title": hit.get("title", ""),
                            "url": url,
                            "domain": domain,
                            "score": hit.get("points", 0),
                            "submitter": hit.get("author", ""),
                            "timestamp": hit.get("created_at", ""),
                            "num_comments": hit.get("num_comments", 0),
                        })

                pbar.set_postfix({"total": len(all_posts)})
                time.sleep(0.2)
                
            except Exception as e:
                print(f"Error: {e}")
                break

        if len(all_posts) >= target_count:
            break

    return all_posts[:target_count]

print("✓ Data fetching function defined (time-based pagination)")

In [ ]:
# Fetch posts from each category
print("Fetching HN posts...")
print("-" * 50)

# Hits: posts with score >= 100
print(f"Fetching {POSTS_PER_CATEGORY:,} hit posts (score >= {MIN_SCORE_HIT})...")
hits = fetch_hn_posts(min_points=MIN_SCORE_HIT, max_points=None, target_count=POSTS_PER_CATEGORY, months_back=24)
print(f"  Got {len(hits):,} hit posts")

# Misses: posts with score 0-9
print(f"Fetching {POSTS_PER_CATEGORY:,} miss posts (score 0-9)...")
misses = fetch_hn_posts(min_points=0, max_points=9, target_count=POSTS_PER_CATEGORY, months_back=24)
print(f"  Got {len(misses):,} miss posts")

# Medium: posts with score 10-50 (for calibration diversity)
print(f"Fetching {POSTS_PER_CATEGORY:,} medium posts (score 10-50)...")
medium_posts = fetch_hn_posts(min_points=10, max_points=50, target_count=POSTS_PER_CATEGORY, months_back=24)
print(f"  Got {len(medium_posts):,} medium posts")

# Combine and label
for p in hits:
    p["label"] = 1
for p in misses:
    p["label"] = 0
for p in medium_posts:
    p["label"] = 1 if p["score"] >= 30 else 0

all_posts = hits + misses + medium_posts
df = pd.DataFrame(all_posts)
df["timestamp"] = pd.to_datetime(df["timestamp"])

print(f"\n✓ Total posts: {len(df):,}")
print(f"  Hits (label=1): {(df['label']==1).sum():,}")
print(f"  Misses (label=0): {(df['label']==0).sum():,}")

## 4. Feature Engineering

In [ ]:
class FeatureEngineer:
    """Extract features for the stacking meta-learner.
    
    NOTE: Domain/author hit rates removed - they cause data leakage and 
    reduce ROC AUC by making the model ignore transformer semantics.
    """
    
    def __init__(self):
        self.word_hit_rates = {}
        self.global_prior = 0.5
        
    def fit(self, df):
        """Learn statistics from training data."""
        # Global prior
        self.global_prior = df["label"].mean()
        
        # Word-level hit rates (less leaky - aggregates over many posts)
        hit_titles = df[df["label"] == 1]["title"].str.lower()
        miss_titles = df[df["label"] == 0]["title"].str.lower()
        
        hit_words = Counter()
        miss_words = Counter()
        for title in hit_titles:
            hit_words.update(title.split())
        for title in miss_titles:
            miss_words.update(title.split())
        
        for word in set(hit_words.keys()) | set(miss_words.keys()):
            h = hit_words.get(word, 0)
            m = miss_words.get(word, 0)
            if h + m >= 50:  # Higher threshold to reduce overfitting
                self.word_hit_rates[word] = h / (h + m)
        
        return self
    
    def transform(self, df):
        """Extract features for prediction."""
        features = pd.DataFrame(index=df.index)
        
        # Title length features
        features["title_length"] = df["title"].str.len()
        features["title_words"] = df["title"].str.split().str.len()
        features["avg_word_length"] = features["title_length"] / (features["title_words"] + 1)
        
        # Title patterns (content-based, not identity-based)
        features["has_show_hn"] = df["title"].str.contains("Show HN", case=False).astype(int)
        features["has_ask_hn"] = df["title"].str.contains("Ask HN", case=False).astype(int)
        features["has_question"] = df["title"].str.contains(r"\?", regex=True).astype(int)
        features["has_numbers"] = df["title"].str.contains(r"\d+", regex=True).astype(int)
        features["has_year"] = df["title"].str.contains(r"20[12]\d", regex=True).astype(int)
        features["starts_capital"] = df["title"].str[0].str.isupper().astype(int)
        features["all_caps_ratio"] = df["title"].apply(
            lambda t: sum(1 for c in t if c.isupper()) / (len(t) + 1)
        )
        
        # Punctuation features
        features["has_colon"] = df["title"].str.contains(":").astype(int)
        features["has_dash"] = df["title"].str.contains(" - |–|—").astype(int)
        features["has_quotes"] = df["title"].str.contains(r'["\']').astype(int)
        features["has_parens"] = df["title"].str.contains(r"\(.*\)").astype(int)
        
        # Word-level features (aggregated, less leaky)
        def calc_word_score(title):
            words = title.lower().split()
            scores = [self.word_hit_rates.get(w, 0.5) for w in words]
            if not scores:
                return 0.5, 0.5, 0.5
            return np.mean(scores), np.max(scores), np.min(scores)
        
        word_scores = df["title"].apply(calc_word_score)
        features["word_hit_mean"] = word_scores.apply(lambda x: x[0])
        features["word_hit_max"] = word_scores.apply(lambda x: x[1])
        features["word_hit_min"] = word_scores.apply(lambda x: x[2])
        
        # Time features (cyclical encoding) - these are OK, not identity-based
        if "timestamp" in df.columns:
            ts = pd.to_datetime(df["timestamp"])
            features["hour_sin"] = np.sin(2 * np.pi * ts.dt.hour / 24)
            features["hour_cos"] = np.cos(2 * np.pi * ts.dt.hour / 24)
            features["dow_sin"] = np.sin(2 * np.pi * ts.dt.dayofweek / 7)
            features["dow_cos"] = np.cos(2 * np.pi * ts.dt.dayofweek / 7)
            features["is_weekend"] = ts.dt.dayofweek.isin([5, 6]).astype(int)
        else:
            features["hour_sin"] = 0
            features["hour_cos"] = 1
            features["dow_sin"] = 0
            features["dow_cos"] = 1
            features["is_weekend"] = 0
        
        # URL type features (category, not identity)
        features["has_url"] = (df["url"].str.len() > 0).astype(int)
        features["is_github"] = df["url"].str.contains("github", case=False).fillna(False).astype(int)
        features["is_medium"] = df["url"].str.contains(r"medium\.com", case=False, regex=True).fillna(False).astype(int)
        features["is_twitter"] = df["url"].str.contains(r"twitter|x\.com", case=False, regex=True).fillna(False).astype(int)
        features["is_youtube"] = df["url"].str.contains(r"youtube|youtu\.be", case=False, regex=True).fillna(False).astype(int)
        features["is_arxiv"] = df["url"].str.contains(r"arxiv\.org", case=False, regex=True).fillna(False).astype(int)
        
        return features.fillna(0)

print("✓ FeatureEngineer class defined (v3.1 - no domain/author leakage)")
print("  Removed: domain_hit_rate, author_hit_rate, domain_count, author_count")
print("  Kept: title patterns, word scores, time features, URL categories")

In [ ]:
# Train/validation/test split
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=SEED, stratify=df["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED, stratify=temp_df["label"])

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}")

# Fit feature engineer on training data
feature_engineer = FeatureEngineer()
feature_engineer.fit(train_df)

# Transform all sets
train_features = feature_engineer.transform(train_df)
val_features = feature_engineer.transform(val_df)
test_features = feature_engineer.transform(test_df)

print(f"\n✓ Engineered {train_features.shape[1]} features")
print(f"Features: {list(train_features.columns)}")

## 5. Focal Loss & RoBERTa Training

In [ ]:
class FocalLoss(nn.Module):
    """
    Focal Loss: FL(p_t) = -alpha_t * (1 - p_t)^gamma * log(p_t)
    
    Key insight: gamma > 0 down-weights easy examples, focusing on hard ones.
    alpha < 0.5 penalizes false positives more (we want fewer FP).
    """
    
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, logits, targets):
        probs = torch.softmax(logits, dim=-1)
        targets_one_hot = F.one_hot(targets, num_classes=2).float()
        
        # Get probability for correct class
        pt = (probs * targets_one_hot).sum(dim=-1)
        
        # Alpha weighting (alpha for positive class, 1-alpha for negative)
        alpha_t = self.alpha * targets.float() + (1 - self.alpha) * (1 - targets.float())
        
        # Focal term: (1 - pt)^gamma
        focal_weight = (1 - pt) ** self.gamma
        
        # Cross entropy
        ce_loss = F.cross_entropy(logits, targets, reduction='none')
        
        # Combined focal loss
        loss = alpha_t * focal_weight * ce_loss
        
        return loss.mean()

print(f"✓ FocalLoss defined (alpha={FOCAL_ALPHA}, gamma={FOCAL_GAMMA})")

In [ ]:
class FocalLossTrainer(Trainer):
    """Custom trainer using Focal Loss instead of CrossEntropy."""
    
    def __init__(self, *args, focal_alpha=0.25, focal_gamma=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss = FocalLoss(alpha=focal_alpha, gamma=focal_gamma)
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.focal_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

print("✓ FocalLossTrainer defined")

In [ ]:
# Tokenize data for RoBERTa
print("Tokenizing data...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_data(texts, labels):
    tokens = tokenizer(
        texts.tolist(),
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_tensors=None
    )
    tokens["labels"] = labels.tolist()
    return Dataset.from_dict(tokens)

train_dataset = tokenize_data(train_df["title"], train_df["label"])
val_dataset = tokenize_data(val_df["title"], val_df["label"])
test_dataset = tokenize_data(test_df["title"], test_df["label"])

train_labels = np.array(train_df["label"])
val_labels = np.array(val_df["label"])
test_labels = np.array(test_df["label"])

print(f"✓ Tokenized: train={len(train_dataset)}, val={len(val_dataset)}, test={len(test_dataset)}")

In [ ]:
# Train RoBERTa with Focal Loss
print("Training RoBERTa with Focal Loss...")
print(f"Focal Loss: alpha={FOCAL_ALPHA}, gamma={FOCAL_GAMMA}")

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

training_args = TrainingArguments(
    output_dir="./roberta_focal",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_steps=100,
    fp16=True,
    seed=SEED,
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.softmax(torch.tensor(logits), dim=-1)[:, 1].numpy()
    preds = (probs >= 0.5).astype(int)
    return {
        "accuracy": accuracy_score(labels, preds),
        "roc_auc": roc_auc_score(labels, probs),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds)
    }

trainer = FocalLossTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    focal_alpha=FOCAL_ALPHA,
    focal_gamma=FOCAL_GAMMA
)

trainer.train()
print("\n✓ RoBERTa training complete")

In [ ]:
# Extract RoBERTa predictions for stacking
def get_roberta_predictions(dataset, batch_size=64):
    """Get probability predictions from RoBERTa."""
    model.eval()
    probs = []
    
    # Use HuggingFace's built-in batching
    dataset_no_labels = dataset.remove_columns(["labels"])
    dataset_no_labels.set_format("torch")
    
    with torch.no_grad():
        for i in tqdm(range(0, len(dataset_no_labels), batch_size), desc="RoBERTa predictions"):
            batch = dataset_no_labels[i:i+batch_size]
            inputs = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**inputs)
            batch_probs = torch.softmax(outputs.logits, dim=-1)[:, 1].cpu().numpy()
            probs.extend(batch_probs)
    
    return np.array(probs)

train_roberta_probs = get_roberta_predictions(train_dataset)
val_roberta_probs = get_roberta_predictions(val_dataset)
test_roberta_probs = get_roberta_predictions(test_dataset)

print(f"\n✓ RoBERTa predictions extracted")
print(f"  Train ROC AUC: {roc_auc_score(train_labels, train_roberta_probs):.4f}")
print(f"  Val ROC AUC:   {roc_auc_score(val_labels, val_roberta_probs):.4f}")
print(f"  Test ROC AUC:  {roc_auc_score(test_labels, test_roberta_probs):.4f}")

## 6. TF-IDF Model

In [ ]:
# Train TF-IDF + Logistic Regression
print("Training TF-IDF model...")

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=5)
train_tfidf = tfidf.fit_transform(train_df["title"])
val_tfidf = tfidf.transform(val_df["title"])
test_tfidf = tfidf.transform(test_df["title"])

tfidf_model = LogisticRegression(max_iter=1000, class_weight="balanced", C=0.5, random_state=SEED)
tfidf_model.fit(train_tfidf, train_labels)

train_tfidf_probs = tfidf_model.predict_proba(train_tfidf)[:, 1]
val_tfidf_probs = tfidf_model.predict_proba(val_tfidf)[:, 1]
test_tfidf_probs = tfidf_model.predict_proba(test_tfidf)[:, 1]

print(f"\n✓ TF-IDF model trained")
print(f"  Train ROC AUC: {roc_auc_score(train_labels, train_tfidf_probs):.4f}")
print(f"  Val ROC AUC:   {roc_auc_score(val_labels, val_tfidf_probs):.4f}")
print(f"  Test ROC AUC:  {roc_auc_score(test_labels, test_tfidf_probs):.4f}")

## 7. LightGBM Stacking Meta-Learner

In [ ]:
# Combine all features for stacking
def build_stacking_features(roberta_probs, tfidf_probs, engineered_features):
    """Combine all feature sources for the meta-learner."""
    return np.column_stack([
        roberta_probs,
        tfidf_probs,
        engineered_features.values
    ])

X_train_stack = build_stacking_features(train_roberta_probs, train_tfidf_probs, train_features)
X_val_stack = build_stacking_features(val_roberta_probs, val_tfidf_probs, val_features)
X_test_stack = build_stacking_features(test_roberta_probs, test_tfidf_probs, test_features)

feature_names = ["roberta_prob", "tfidf_prob"] + list(train_features.columns)

print(f"✓ Stacking features: {X_train_stack.shape[1]} dimensions")
print(f"  - RoBERTa prob: 1")
print(f"  - TF-IDF prob: 1")
print(f"  - Engineered: {train_features.shape[1]}")

In [ ]:
# Train LightGBM stacking meta-learner
print("Training LightGBM stacking meta-learner...")
print(f"Parameters: {LGBM_PARAMS}")

lgbm_train = lgb.Dataset(X_train_stack, label=train_labels, feature_name=feature_names)
lgbm_val = lgb.Dataset(X_val_stack, label=val_labels, feature_name=feature_names, reference=lgbm_train)

stacker = lgb.train(
    LGBM_PARAMS,
    lgbm_train,
    num_boost_round=500,
    valid_sets=[lgbm_train, lgbm_val],
    valid_names=["train", "val"],
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(50)]
)

# Get raw predictions (before calibration)
train_stacker_raw = stacker.predict(X_train_stack)
val_stacker_raw = stacker.predict(X_val_stack)
test_stacker_raw = stacker.predict(X_test_stack)

print(f"\n✓ LightGBM Stacker trained")
print(f"  Best iteration: {stacker.best_iteration}")
print(f"  Val ROC AUC (raw): {roc_auc_score(val_labels, val_stacker_raw):.4f}")
print(f"  Test ROC AUC (raw): {roc_auc_score(test_labels, test_stacker_raw):.4f}")

## 8. Isotonic Calibration

In [ ]:
# Apply Isotonic Regression calibration
print("Applying Isotonic Regression calibration...")

calibrator = IsotonicRegression(out_of_bounds="clip")
calibrator.fit(val_stacker_raw, val_labels)

# Calibrated predictions
train_probs = calibrator.predict(train_stacker_raw)
val_probs = calibrator.predict(val_stacker_raw)
test_probs = calibrator.predict(test_stacker_raw)

# Calculate calibration improvement
def calculate_ece(probs, labels, n_bins=10):
    """Calculate Expected Calibration Error."""
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        in_bin = (probs > bin_boundaries[i]) & (probs <= bin_boundaries[i+1])
        if in_bin.sum() > 0:
            bin_accuracy = labels[in_bin].mean()
            bin_confidence = probs[in_bin].mean()
            ece += (in_bin.sum() / len(probs)) * abs(bin_accuracy - bin_confidence)
    return ece

ece_before = calculate_ece(test_stacker_raw, test_labels)
ece_after = calculate_ece(test_probs, test_labels)

print(f"\n✓ Isotonic calibration applied")
print(f"  ECE before calibration: {ece_before:.4f}")
print(f"  ECE after calibration: {ece_after:.4f}")
print(f"  Improvement: {((ece_before - ece_after) / ece_before * 100):.1f}%")

## 9. Threshold Optimization & Evaluation

In [ ]:
# Optimize classification threshold
print("Optimizing classification threshold...")

precisions, recalls, thresholds = precision_recall_curve(test_labels, test_probs)

# Find threshold that maximizes F1
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-8)
best_f1_idx = np.argmax(f1_scores)
best_f1_threshold = thresholds[best_f1_idx] if best_f1_idx < len(thresholds) else 0.5

# Find threshold for balanced precision/recall
balanced_diff = np.abs(precisions[:-1] - recalls[:-1])
balanced_idx = np.argmin(balanced_diff)
balanced_threshold = thresholds[balanced_idx]

# Find threshold for target precision (0.6)
target_precision = 0.6
valid_mask = recalls[:-1] > 0.3
precision_diff = np.abs(precisions[:-1] - target_precision)
precision_diff[~valid_mask] = np.inf
target_idx = np.argmin(precision_diff)
precision_threshold = thresholds[target_idx]

print(f"\nThreshold Analysis:")
print(f"  Best F1 threshold: {best_f1_threshold:.3f}")
print(f"    -> F1={f1_scores[best_f1_idx]:.3f}, P={precisions[best_f1_idx]:.3f}, R={recalls[best_f1_idx]:.3f}")
print(f"  Balanced P/R threshold: {balanced_threshold:.3f}")
print(f"    -> P={precisions[balanced_idx]:.3f}, R={recalls[balanced_idx]:.3f}")
print(f"  Precision {target_precision:.0%} threshold: {precision_threshold:.3f}")
print(f"    -> P={precisions[target_idx]:.3f}, R={recalls[target_idx]:.3f}")

# Use balanced threshold as default
OPTIMAL_THRESHOLD = balanced_threshold
print(f"\n✓ Selected threshold: {OPTIMAL_THRESHOLD:.3f}")

In [ ]:
# Full evaluation report
print("=" * 60)
print("V3 STACKING META-LEARNER - FULL EVALUATION REPORT")
print("=" * 60)

test_preds = (test_probs >= OPTIMAL_THRESHOLD).astype(int)

# Core metrics
roc_auc = roc_auc_score(test_labels, test_probs)
avg_precision = average_precision_score(test_labels, test_probs)
precision = precision_score(test_labels, test_preds)
recall = recall_score(test_labels, test_preds)
f1 = f1_score(test_labels, test_preds)
accuracy = accuracy_score(test_labels, test_preds)

print(f"\n📊 CORE METRICS (threshold={OPTIMAL_THRESHOLD:.3f}):")
print(f"  ROC AUC:           {roc_auc:.4f}")
print(f"  Average Precision: {avg_precision:.4f}")
print(f"  Precision:         {precision:.4f}")
print(f"  Recall:            {recall:.4f}")
print(f"  F1 Score:          {f1:.4f}")
print(f"  Accuracy:          {accuracy:.4f}")

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, test_preds).ravel()
print(f"\n📋 CONFUSION MATRIX:")
print(f"  True Negatives:  {tn:,}")
print(f"  False Positives: {fp:,}")
print(f"  False Negatives: {fn:,}")
print(f"  True Positives:  {tp:,}")
print(f"  FP Rate: {fp/(fp+tn):.2%}")
print(f"  FN Rate: {fn/(fn+tp):.2%}")

# Calibration
print(f"\n🎯 CALIBRATION:")
print(f"  ECE (Expected Calibration Error): {ece_after:.4f}")

# Comparison with v2
print(f"\n📈 COMPARISON WITH V2:")
v2_roc = 0.6984
v2_precision = 0.4173
v2_fp = 2847
v2_ece = 0.1135
print(f"  ROC AUC:   {v2_roc:.4f} -> {roc_auc:.4f} ({'↑' if roc_auc > v2_roc else '↓'} {abs(roc_auc - v2_roc):.4f})")
print(f"  Precision: {v2_precision:.4f} -> {precision:.4f} ({'↑' if precision > v2_precision else '↓'} {abs(precision - v2_precision):.4f})")
print(f"  FP Count:  {v2_fp:,} -> {fp:,} ({'↓' if fp < v2_fp else '↑'} {abs(fp - v2_fp):,})")
print(f"  ECE:       {v2_ece:.4f} -> {ece_after:.4f} ({'↓' if ece_after < v2_ece else '↑'} {abs(ece_after - v2_ece):.4f})")

print("\n" + "=" * 60)

In [ ]:
# Feature importance analysis
print("Feature Importance Analysis (LightGBM Stacker):")
print("-" * 50)

importance = stacker.feature_importance(importance_type="gain")
importance_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importance
}).sort_values("importance", ascending=False)

print("\nTop 15 Most Important Features:")
for _, row in importance_df.head(15).iterrows():
    bar = "█" * int(row["importance"] / importance_df["importance"].max() * 20)
    print(f"  {row['feature'][:25]:<25} {bar} {row['importance']:.1f}")

# Model contribution analysis
print("\n\nModel Contribution Analysis:")
print("-" * 50)
roberta_imp = importance_df[importance_df["feature"] == "roberta_prob"]["importance"].values[0]
tfidf_imp = importance_df[importance_df["feature"] == "tfidf_prob"]["importance"].values[0]
eng_imp = importance_df[~importance_df["feature"].isin(["roberta_prob", "tfidf_prob"])]["importance"].sum()
total_imp = roberta_imp + tfidf_imp + eng_imp

print(f"  RoBERTa probability:    {roberta_imp/total_imp*100:5.1f}%")
print(f"  TF-IDF probability:     {tfidf_imp/total_imp*100:5.1f}%")
print(f"  Engineered features:    {eng_imp/total_imp*100:5.1f}%")

In [ ]:
# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. ROC Curve
ax1 = axes[0, 0]
fpr, tpr, _ = roc_curve(test_labels, test_probs)
ax1.plot(fpr, tpr, "b-", linewidth=2, label=f"V3 Stacker (AUC={roc_auc:.3f})")
ax1.plot([0, 1], [0, 1], "k--", alpha=0.5)
ax1.fill_between(fpr, tpr, alpha=0.2)
ax1.set_xlabel("False Positive Rate")
ax1.set_ylabel("True Positive Rate")
ax1.set_title("ROC Curve - V3 Stacking Meta-Learner")
ax1.legend(loc="lower right")
ax1.grid(True, alpha=0.3)

# 2. Precision-Recall Curve
ax2 = axes[0, 1]
ax2.plot(recalls, precisions[:-1], "g-", linewidth=2, label=f"V3 (AP={avg_precision:.3f})")
ax2.axhline(y=0.5, color="r", linestyle="--", alpha=0.5, label="Random (AP=0.5)")
ax2.scatter([recall], [precision], color="red", s=100, zorder=5, label=f"Operating point")
ax2.set_xlabel("Recall")
ax2.set_ylabel("Precision")
ax2.set_title("Precision-Recall Curve")
ax2.legend(loc="upper right")
ax2.grid(True, alpha=0.3)

# 3. Calibration Plot
ax3 = axes[1, 0]
n_bins = 10
bin_means, bin_true = [], []
for i in range(n_bins):
    low, high = i / n_bins, (i + 1) / n_bins
    mask = (test_probs >= low) & (test_probs < high)
    if mask.sum() > 0:
        bin_means.append(test_probs[mask].mean())
        bin_true.append(test_labels[mask].mean())

ax3.plot([0, 1], [0, 1], "k--", label="Perfect calibration")
ax3.scatter(bin_means, bin_true, s=100, alpha=0.7, label="V3 Calibrated")
ax3.plot(bin_means, bin_true, "b-", alpha=0.5)
ax3.set_xlabel("Mean Predicted Probability")
ax3.set_ylabel("Fraction of Positives")
ax3.set_title(f"Calibration Plot (ECE={ece_after:.4f})")
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Confusion Matrix
ax4 = axes[1, 1]
cm = confusion_matrix(test_labels, test_preds)
sns.heatmap(cm, annot=True, fmt=",", cmap="Blues", ax=ax4,
            xticklabels=["Predicted 0", "Predicted 1"],
            yticklabels=["Actual 0", "Actual 1"])
ax4.set_title(f"Confusion Matrix (threshold={OPTIMAL_THRESHOLD:.2f})")

plt.tight_layout()
plt.savefig("v3_evaluation_plots.png", dpi=150, bbox_inches="tight")
plt.show()
print("\n✓ Plots saved to v3_evaluation_plots.png")

## 10. Demo & Save Model

In [ ]:
# Demo predictions on sample titles
print("Demo Predictions on Sample Titles:")
print("=" * 70)

sample_titles = [
    "Show HN: I built a real-time collaborative code editor",
    "Why I'm mass-deleting my old tweets",
    "The hidden costs of serverless architectures",
    "Ask HN: What are you working on this weekend?",
    "GPT-5 achieves human-level performance on coding benchmarks",
    "How we reduced our AWS bill by 80%",
    "A random blog post about my cat",
    "The Rust programming language is overrated",
    "New research shows coffee may extend lifespan",
    "I accidentally deleted production database, here's what happened"
]

# Create minimal dataframe for prediction
sample_df = pd.DataFrame({"title": sample_titles})
sample_df["url"] = ""
sample_df["submitter"] = "unknown"
sample_df["timestamp"] = pd.Timestamp.now()

# Get features for samples
sample_tokens = tokenizer(sample_titles, truncation=True, padding="max_length", 
                          max_length=MAX_LENGTH, return_tensors="pt")

# RoBERTa predictions
model.eval()
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in sample_tokens.items()}
    outputs = model(**inputs)
    sample_roberta_probs = torch.softmax(outputs.logits, dim=-1)[:, 1].cpu().numpy()

# TF-IDF predictions
sample_tfidf_probs = tfidf_model.predict_proba(tfidf.transform(sample_titles))[:, 1]

# Engineered features
sample_eng = feature_engineer.transform(sample_df)

# Stack and predict
sample_stack = build_stacking_features(sample_roberta_probs, sample_tfidf_probs, sample_eng)
sample_raw = stacker.predict(sample_stack)
sample_probs = calibrator.predict(sample_raw)

print(f"{'Title':<55} {'Prob':>6} {'Pred':>6}")
print("-" * 70)
for title, prob in zip(sample_titles, sample_probs):
    pred = "✅ HIT" if prob >= OPTIMAL_THRESHOLD else "❌ MISS"
    title_short = title[:52] + "..." if len(title) > 55 else title
    print(f"{title_short:<55} {prob:>5.1%} {pred:>6}")

In [ ]:
# Save all model components
print("Saving model components...")
os.makedirs("./hn_model_v3", exist_ok=True)

# Save RoBERTa model
model.save_pretrained("./hn_model_v3/roberta")
tokenizer.save_pretrained("./hn_model_v3/roberta")
print("✓ RoBERTa model saved")

# Save TF-IDF + LogReg
joblib.dump(tfidf, "./hn_model_v3/tfidf_vectorizer.joblib")
joblib.dump(tfidf_model, "./hn_model_v3/tfidf_model.joblib")
print("✓ TF-IDF model saved")

# Save LightGBM stacker
stacker.save_model("./hn_model_v3/lgbm_stacker.txt")
print("✓ LightGBM stacker saved")

# Save calibrator
joblib.dump(calibrator, "./hn_model_v3/isotonic_calibrator.joblib")
print("✓ Isotonic calibrator saved")

# Save feature engineer
joblib.dump(feature_engineer, "./hn_model_v3/feature_engineer.joblib")
print("✓ Feature engineer saved")

# Save configuration
config = {
    "version": "v3",
    "architecture": "RoBERTa + TF-IDF + Engineered Features -> LightGBM Stacker -> Isotonic Calibration",
    "optimal_threshold": float(OPTIMAL_THRESHOLD),
    "focal_loss": {"alpha": FOCAL_ALPHA, "gamma": FOCAL_GAMMA},
    "lgbm_params": LGBM_PARAMS,
    "metrics": {
        "roc_auc": float(roc_auc),
        "average_precision": float(avg_precision),
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1),
        "ece": float(ece_after)
    },
    "confusion_matrix": {"tn": int(tn), "fp": int(fp), "fn": int(fn), "tp": int(tp)},
    "feature_names": feature_names
}

with open("./hn_model_v3/config.json", "w") as f:
    json.dump(config, f, indent=2)
print("✓ Configuration saved")

# Create zip for download
!cd hn_model_v3 && zip -r ../hn_model_v3.zip .
print("\n✓ All components saved to hn_model_v3.zip")

# List files
!ls -la hn_model_v3/

In [ ]:
# Download the model zip
from google.colab import files
files.download("hn_model_v3.zip")